In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [34]:
# 1. 生成一些简单的数据
# 假设我们有一个二分类问题，输入特征维度为10
torch.manual_seed(42)  # 设置随机种子以保证可重复性
X = torch.randn(1000, 10)  # 1000个样本，每个样本有10个特征
y = (X[:, 0] * X[:, 1] + X[:, 2] - X[:, 3] > 0).float()  # 复杂的二分类标签

In [46]:
print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: torch.Size([1000, 10])
y shape: torch.Size([1000])


In [47]:
# 2. 将数据封装成DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [48]:
# 3. 定义DNN模型
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(10, 64)  # 输入层到第一个隐藏层，10个输入特征，64个隐藏单元
        self.fc2 = nn.Linear(64, 32)  # 第一个隐藏层到第二个隐藏层，64个隐藏单元，32个隐藏单元
        self.fc3 = nn.Linear(32, 16)  # 第二个隐藏层到第三个隐藏层，32个隐藏单元，16个隐藏单元
        self.fc4 = nn.Linear(16, 1)   # 第三个隐藏层到输出层，16个隐藏单元，1个输出单元
        self.sigmoid = nn.Sigmoid()   # 输出层的激活函数

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # 第一层全连接 + ReLU激活
        x = torch.relu(self.fc2(x))  # 第二层全连接 + ReLU激活
        x = torch.relu(self.fc3(x))  # 第三层全连接 + ReLU激活
        x = self.sigmoid(self.fc4(x))  # 输出层全连接 + Sigmoid激活
        return x

In [37]:
# 4. 初始化模型、损失函数和优化器
model = DNN()
criterion = nn.BCELoss()  # 二分类交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam优化器

In [38]:
# 5. 训练模型
num_epochs = 20
for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        # 前向传播
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)

        # 反向传播和优化
        optimizer.zero_grad()  # 清空梯度
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新参数

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.6767
Epoch [2/20], Loss: 0.5479
Epoch [3/20], Loss: 0.2509
Epoch [4/20], Loss: 0.3119
Epoch [5/20], Loss: 0.3064
Epoch [6/20], Loss: 0.2931
Epoch [7/20], Loss: 0.1275
Epoch [8/20], Loss: 0.0471
Epoch [9/20], Loss: 0.1795
Epoch [10/20], Loss: 0.2225
Epoch [11/20], Loss: 0.0774
Epoch [12/20], Loss: 0.2450
Epoch [13/20], Loss: 0.0966
Epoch [14/20], Loss: 0.0024
Epoch [15/20], Loss: 0.0081
Epoch [16/20], Loss: 0.3219
Epoch [17/20], Loss: 0.0208
Epoch [18/20], Loss: 0.0651
Epoch [19/20], Loss: 0.0207
Epoch [20/20], Loss: 0.0014


In [39]:
# 6. 测试模型
with torch.no_grad():
    test_X = torch.randn(5, 10)  # 生成5个测试样本
    predictions = model(test_X)
    print("Predictions:", predictions)

Predictions: tensor([[9.9870e-01],
        [1.0498e-03],
        [2.6688e-03],
        [9.8690e-05],
        [5.5142e-07]])
